In [19]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, optimizers, backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.linear_model import LogisticRegression
import pickle
import os

In [ ]:
df = pd.read_excel('datasets/donnees_nettoyees.xlsx')

: 

In [5]:
print("Fichier chargé !")
print(f"Shape : {df.shape}")
print("\nColonnes présentes :")
print(df.columns.tolist())

Fichier chargé !
Shape : (569, 32)

Colonnes présentes :
['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']


In [6]:
X = df.drop(['id', 'diagnosis'], axis=1)
y = df['diagnosis']
# Afficher les dimensions
print("Shape X:", X.shape)
print("Shape y:", y.shape)

Shape X: (569, 30)
Shape y: (569,)


In [7]:
print(df['diagnosis'].value_counts())

diagnosis
0    357
1    212
Name: count, dtype: int64


In [8]:
# Découpage 70% train et 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,   # 30% pour le test
    random_state=42,  # pour reproductibilité
    stratify=y       # pour garder la proportion des classes
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


def evaluate(y_true, y_pred, algo_name=None, k=None, distance=None, epochs=None, return_dict=False):
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    tpr = tp / (tp + fn) if (tp + fn) else 0
    tnr = tn / (tn + fp) if (tn + fp) else 0
    fpr = fp / (tn + fp) if (tn + fp) else 0
    fnr = fn / (tp + fn) if (tp + fn) else 0

    print("\n══════════════════════════════════════════")
    print(f"📌 Model : {algo_name if algo_name else 'Model'}")
    print("══════════════════════════════════════════")
    print(f"Accuracy : {acc*100:.6f}%")
    print(f"TPR (Sensitivity) : {tpr*100:.6f}%")
    print(f"TNR (Specificity) : {tnr*100:.6f}%")
    print(f"FPR : {fpr*100:.6f}%")
    print(f"FNR : {fnr*100:.6f}%")
    print("Confusion matrix:")
    print(f"[[{tn} {fp}]")
    print(f" [{fn} {tp}]]")
    print("══════════════════════════════════════════")

    if return_dict:
        if k is not None:
            return {
                "Algorithm": algo_name,
                "Accuracy %": round(acc * 100, 6),
                "k": k,
                "Distance": distance,
                "FPR %": round(fpr * 100, 6),
                "FNR %": round(fnr * 100, 6),
                "TPR %": round(tpr * 100, 6),
                "TNR %": round(tnr * 100, 6),
            }
        return {
            "Algorithm": algo_name,
            "Accuracy %": round(acc * 100, 6),
            "Data points": len(y_true),
            "Epochs": epochs,
            "FPR %": round(fpr * 100, 6),
            "FNR %": round(fnr * 100, 6),
            "TPR %": round(tpr * 100, 6),
            "TNR %": round(tnr * 100, 6),
        }

1. GRU-SVM (Batch 128, LR=1e-3, Epochs=3000, L2, C=5)


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, backend as K

y_train_s = (y_train * 2 - 1).astype(float)
y_test_s = (y_test * 2 - 1).astype(float)

input_dim = X_train.shape[1]
Xtr = X_train.reshape(-1, 1, input_dim)
Xte = X_test.reshape(-1, 1, input_dim)


def squared_hinge_l2(y_true, y_pred):
    margin = 1 - y_true * y_pred
    # C = 5
    return K.mean(K.square(K.maximum(0.0, margin))) + 5 * K.sum(K.square(y_pred))


gru_svm_model = models.Sequential([
    layers.Input(shape=(1, input_dim)),
    # Cell size=128, dropout=0.5
    layers.GRU(128, return_sequences=False, dropout=0.5),
    layers.Dense(1, activation='linear')
])

gru_svm_model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
                      loss=squared_hinge_l2)

gru_svm_model.fit(Xtr, y_train_s, epochs=3000, batch_size=128, verbose=0)

y_out = gru_svm_model.predict(Xte).flatten()
y_pred = (np.sign(y_out) > 0).astype(int)

print("\n=== GRU-SVM ===")
evaluate(y_test, y_pred, algo_name="GRU-SVM ")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

=== GRU-SVM ===

══════════════════════════════════════════
📌 Model : GRU-SVM 
══════════════════════════════════════════
Accuracy : 97.660819%
TPR (Sensitivity) : 95.312500%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 4.687500%
Confusion matrix:
[[106 1]
 [3 61]]
══════════════════════════════════════════


2. SVM (Batch 128, LR=1e-3, Epochs=3000, L2, C=5)


In [10]:
# 1. Conversion y → -1 / +1 (nécessaire pour la hinge loss)
y_train_svm = np.where(y_train == 0, -1.0, 1.0)
y_test_svm = np.where(y_test == 0, -1.0, 1.0)

# 2. Hyperparamètres pour svm
batch_size = 128
learning_rate = 1e-3  # 0.001
epochs = 3000
C = 5

# 3. Fonction de perte personnalisée


def l2_svm_loss(y_true, y_pred):
    hinge = tf.maximum(1.0 - y_true * y_pred, 0.0)
    squared_hinge = tf.square(hinge)
    return C * tf.reduce_mean(squared_hinge)


# 4. Modèle linéaire = SVM linéaire
l2_svm_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(
        1,
        use_bias=True,
        kernel_regularizer=tf.keras.regularizers.l2(1.0 / (2 * C))  # 0.1
    )
])
# 5. Compilation avec Adam
l2_svm_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=l2_svm_loss,
    metrics=[]
)

# 6. Entraînement avec early stopping
callbacks = [
    EarlyStopping(monitor='loss', patience=300,
                  restore_best_weights=True, verbose=1)
]

history = l2_svm_model.fit(
    X_train, y_train_svm,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks,
    shuffle=True
)
# 7. Prédictions
y_pred_raw = l2_svm_model.predict(X_test, verbose=0)
y_pred_svm = np.sign(y_pred_raw).flatten()
y_pred = np.where(y_pred_svm == -1, 0, 1)

# 8. Évaluation
evaluate(
    y_true=y_test,
    y_pred=y_pred,
    algo_name="L2-SVM",
    epochs=3000,
    return_dict=False
)

Epoch 1/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25.8540  
Epoch 2/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24.7818 
Epoch 3/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 23.7319 
Epoch 4/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 22.7529 
Epoch 5/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21.7939 
Epoch 6/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 20.8704 
Epoch 7/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 19.9862 
Epoch 8/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 19.1159
Epoch 9/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 18.3242 
Epoch 10/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - loss: 17.5367
Epoch 11/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 16.8056 
Epoch 12/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 16.1132
Epoch 13/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15.4414 
Epoch 14/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.8246 
Epoch 15/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - lo

3. MLP (Arch: 500-500-500, Act=ReLU, Opt=SGD, Loss=BinaryCE, Batch=128, Epochs=3000)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# Création du modèle MLP
mlp = Sequential([
    Dense(500, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(500, activation='relu'),
    Dense(500, activation='relu'),
    Dense(1, activation='sigmoid')  # sortie binaire
])

# Optimiseur SGD
optimizer = SGD(learning_rate=0.01)

mlp.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Définir les epochs
EPOCHS_MLP = 3000

# Entraînement
history_mlp = mlp.fit(
    X_train, y_train,
    epochs=EPOCHS_MLP,
    batch_size=128,
    verbose=0
)

# Prédiction
y_pred_mlp = (mlp.predict(X_test) > 0.5).astype(int)

# Évaluation
acc_mlp = accuracy_score(y_test, y_pred_mlp)
cm_mlp = confusion_matrix(y_test, y_pred_mlp)

evaluate(y_test, y_pred_mlp, algo_name="MLP", epochs=EPOCHS_MLP)

C:\Users\Ahmed-Trabelsi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

══════════════════════════════════════════
📌 Model : MLP
══════════════════════════════════════════
Accuracy : 98.245614%
TPR (Sensitivity) : 95.312500%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 4.687500%
Confusion matrix:
[[107 0]
 [3 61]]
══════════════════════════════════════════


soft-max Régression


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import numpy as np

# One-hot encoding pour classification binaire (2 classes)
y_train_softmax = to_categorical(y_train, num_classes=2)
y_test_softmax = to_categorical(y_test, num_classes=2)

# Modèle de Régression Softmax (simple couche linéaire + softmax)
softmax_model = Sequential([
    # 2 sorties pour classes binaires
    Dense(2, input_shape=(X_train.shape[1],), activation='softmax')
])

# Compilation avec categorical_crossentropy (perte pour softmax)
softmax_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Entraînement (équivalent à softmax regression)
softmax_model.fit(X_train, y_train_softmax,
                  epochs=100, batch_size=32, verbose=0)

# Prédictions (argmax pour obtenir les classes)
y_pred_softmax = np.argmax(softmax_model.predict(X_test), axis=1)

# Évaluation directe (pour vérifier l'accuracy ~99%)
evaluate(y_test, y_pred_softmax, algo_name="Softmax Regression")

C:\Users\Ahmed-Trabelsi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

══════════════════════════════════════════
📌 Model : Softmax Regression
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 6.250000%
Confusion matrix:
[[106 1]
 [4 60]]
══════════════════════════════════════════


In [13]:
base_tree = DecisionTreeClassifier(
    max_depth=2,         # faible profondeur = meilleur généralisation
    min_samples_split=5
)

ada = AdaBoostClassifier(
    estimator=base_tree,
    n_estimators=300,        # nombre de weak learners
    learning_rate=0.5,       # vitesse d'apprentissage
    algorithm='SAMME',     # meilleur pour outputs probabilistes
    random_state=42
)

ada.fit(X_train, y_train)
ada_pred = ada.predict(X_test)

evaluate(y_test, ada_pred, algo_name="AdaBoost")

C:\Users\Ahmed-Trabelsi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(



══════════════════════════════════════════
📌 Model : AdaBoost
══════════════════════════════════════════
Accuracy : 97.660819%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 6.250000%
Confusion matrix:
[[107 0]
 [4 60]]
══════════════════════════════════════════


In [14]:
# Régression linéaire pour classification binaire
from sklearn.linear_model import LinearRegression

# Entraînement
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Prédiction (on arrondit à 0 ou 1)
y_pred_linreg = (lin_reg.predict(X_test) >= 0.5).astype(int)

# Évaluation
evaluate(y_test, y_pred_linreg, algo_name="Linear Regression")


══════════════════════════════════════════
📌 Model : Linear Regression
══════════════════════════════════════════
Accuracy : 95.321637%
TPR (Sensitivity) : 87.500000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 12.500000%
Confusion matrix:
[[107 0]
 [8 56]]
══════════════════════════════════════════


In [15]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.1, 0.5, 1.0],
    'estimator__max_depth': [1, 2, 3]
}

grid = GridSearchCV(
    AdaBoostClassifier(estimator=DecisionTreeClassifier(), random_state=42),
    param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
best_model = grid.best_estimator_
ada_pred = best_model.predict(X_test)
evaluate(y_test, ada_pred, algo_name="AdaBoost Tuned")

Best params: {'estimator__max_depth': 3, 'learning_rate': 1.0, 'n_estimators': 200}

══════════════════════════════════════════
📌 Model : AdaBoost Tuned
══════════════════════════════════════════
Accuracy : 97.660819%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 6.250000%
Confusion matrix:
[[107 0]
 [4 60]]
══════════════════════════════════════════


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Entraîner la Régression Logistique
linreg_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs'
)

linreg_model.fit(X_train, y_train)
log_reg_pred = linreg_model.predict(X_test)

evaluate(y_test, log_reg_pred, algo_name="Logistic Regression")


══════════════════════════════════════════
📌 Model : Logistic Regression
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 6.250000%
Confusion matrix:
[[106 1]
 [4 60]]
══════════════════════════════════════════


4. KNN


In [17]:
# 1️⃣ Fonctions de distance
def L1(x1, x2):
    return np.sum(np.abs(x1 - x2))


def L2(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

# 2️⃣ Prédiction pour un seul exemple


def knn_single_predict(X_train, y_train, x_query, k=1, metric="L2"):
    distances = []
    for i in range(len(X_train)):
        dist = L1(x_query, X_train[i]) if metric == "L1" else L2(
            x_query, X_train[i])
        distances.append((dist, y_train[i]))

    # Trier par distance
    distances.sort(key=lambda x: x[0])

    # k voisins
    neighbors = [label for _, label in distances[:k]]

    # Vote majoritaire
    return 1 if neighbors.count(1) > neighbors.count(0) else 0

# 3️⃣ Prédiction pour tout X_test


def knn_predict(X_train, y_train, X_test, k=1, metric="L2"):
    preds = []
    for i in range(len(X_test)):
        preds.append(knn_single_predict(
            X_train, y_train, X_test[i], k=k, metric=metric))
    return np.array(preds)

# 4️⃣ Sélection du meilleur modèle (sans évaluation)


def knn_tune_and_evaluate(X_train, y_train, X_test, y_test):
    k_values = [1, 3, 5, 7]
    distances = ["L1", "L2"]
    results = []

    for k in k_values:
        for dist in distances:
            y_pred = knn_predict(X_train, y_train, X_test, k=k, metric=dist)
            acc = accuracy_score(y_test, y_pred)
            results.append((k, dist, acc, y_pred))

    # Meilleur modèle
    best_k, best_metric, best_acc, best_preds = max(
        results, key=lambda x: x[2])

    print("\n══════════════════════════════════════════")
    print(f"🏆 BEST KNN MODEL SELECTED")
    print("══════════════════════════════════════════")
    print(f"➡️ Best k : {best_k}")
    print(f"➡️ Best distance metric : {best_metric}")
    print(f"➡️ Best accuracy : {best_acc*100:.4f}%")
    print("══════════════════════════════════════════")

    return best_k, best_metric, best_acc, best_preds


# 1️⃣ Trouver le meilleur modèle
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

best_k, best_metric, best_acc, best_preds = knn_tune_and_evaluate(
    X_train, y_train, X_test, y_test)

# 2️⃣ Appeler evaluate_fn séparément
evaluate(
    y_true=y_test,
    y_pred=best_preds,
    algo_name="KNN",
    k=best_k,
    distance="Manhattan (L1)" if best_metric == "L1" else "Euclidean (L2)"
)


══════════════════════════════════════════
🏆 BEST KNN MODEL SELECTED
══════════════════════════════════════════
➡️ Best k : 1
➡️ Best distance metric : L1
➡️ Best accuracy : 96.4912%
══════════════════════════════════════════

══════════════════════════════════════════
📌 Model : KNN
══════════════════════════════════════════
Accuracy : 96.491228%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 98.130841%
FPR : 1.869159%
FNR : 6.250000%
Confusion matrix:
[[105 2]
 [4 60]]
══════════════════════════════════════════


In [18]:
# Comparaison des modèles
results = []

# GRU-SVM
results.append(evaluate(y_test, y_pred, algo_name="GRU-SVM", return_dict=True))

# L2-SVM
results.append(evaluate(y_test, y_pred,
               algo_name="L2-SVM", epochs=3000, return_dict=True))

# Linear Regression
results.append(evaluate(y_test, y_pred_linreg,
               algo_name="Linear Regression", return_dict=True))

# Softmax Regression
results.append(evaluate(y_test, y_pred_softmax,
               algo_name="Softmax Regression", return_dict=True))

# --- KNN ---
results.append(evaluate(
    y_true=y_test,
    y_pred=best_preds,
    algo_name=f"KNN (k={best_k}, distance={'Manhattan' if best_metric=='L1' else 'Euclidean'})",
    return_dict=True
))
results.append(evaluate(y_test, y_pred_mlp, algo_name="MLP",
               epochs=EPOCHS_MLP, return_dict=True))

results.append(evaluate(y_test, ada_pred,
               algo_name="AdaBoost", return_dict=True))

results.append(evaluate(y_test, ada_pred,
               algo_name="AdaBoost Tuned", return_dict=True))

results.append(evaluate(y_test, log_reg_pred,
               algo_name="Logistic Regression", return_dict=True))

# Créer un DataFrame pour comparaison
df_results = pd.DataFrame(results)
print("\n" + "="*80)
print("📊 COMPARAISON FINALE DES MODÈLES")
print("="*80)
print(df_results.to_string(index=False))

# Meilleur modèle
best_algo = df_results.loc[df_results['Accuracy %'].idxmax()]
print("\n🏆 MEILLEUR MODÈLE :")
print(
    f"   {best_algo['Algorithm']} avec une précision de {best_algo['Accuracy %']}%")


# Créer le dossier s'il n'existe pas
os.makedirs("./models", exist_ok=True)

# Dictionnaire des modèles à sauvegarder
models_to_save = {
    "gru_svm.pkl": gru_svm_model,
    "l2_svm.pkl": l2_svm_model,
    "linear_regression.pkl": linreg_model,
    "softmax_regression.pkl": softmax_model
}

# Sauvegarde
for filename, model in models_to_save.items():
    path = os.path.join("./models", filename)
    with open(path, "wb") as f:
        pickle.dump(model, f)
    # print(f"Modèle exporté : {path}")


══════════════════════════════════════════
📌 Model : GRU-SVM
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 6.250000%
Confusion matrix:
[[106 1]
 [4 60]]
══════════════════════════════════════════

══════════════════════════════════════════
📌 Model : L2-SVM
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 6.250000%
Confusion matrix:
[[106 1]
 [4 60]]
══════════════════════════════════════════

══════════════════════════════════════════
📌 Model : Linear Regression
══════════════════════════════════════════
Accuracy : 95.321637%
TPR (Sensitivity) : 87.500000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 12.500000%
Confusion matrix:
[[107 0]
 [8 56]]
══════════════════════════════════════════

══════════════════════════════════════════
📌 Model : Softmax Regression
════════════════